### Requirements Installation

In [5]:
!pip install openai
!pip install langchain
!pip install python-dotenv

In [6]:
!pip show langchain

Name: langchain
Version: 0.1.9
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/ds/Documents/Anaconda/anaconda3/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


### Load Environment Variables

In [7]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

print("API Key Loaded:", os.environ.get('OPENAI_API_KEY') is not None)

API Key Loaded: True


### Create LangChain Agent

In [8]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_experimental.utilities import PythonREPL

In [11]:
python_repl = PythonREPL()

In [13]:
python_repl.run('import datetime; print(datetime.datetime.now())')

'2024-02-24 20:56:17.040768\n'

In [14]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

In [15]:
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

In [16]:
agent = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [17]:
agent.invoke('Calculate the square root of the factorial of 10 and display it with 4 decimal point')



> Entering new AgentExecutor chain...
First, I need to calculate the factorial of 10. Then, I will find the square root of that result. Finally, I will format the result to display it with 4 decimal points. I can use the `math` module in Python to accomplish these tasks. Let's start by calculating the factorial of 10. 

Action: Python_REPL
Action Input: import math
print(math.factorial(10))
Observation: 3628800

Thought:Now that I have the factorial of 10, I need to calculate the square root of this number. I can continue using the `math` module for this.

Action: Python_REPL
Action Input: print(math.sqrt(3628800))
Observation: 1904.9409439665053

Thought:I now have the square root of the factorial of 10. The next step is to format this result to display it with 4 decimal points.

Action: Python_REPL
Action Input: print(f"{1904.9409439665053:.4f}")
Observation: 1904.9409

Thought:I now know the final answer
Final Answer: 1904.9409

> Finished chain.


{'input': 'Calculate the square root of the factorial of 10 and display it with 4 decimal point',
 'output': '1904.9409'}

In [18]:
agent.invoke('What is the asnwer to 3.14 ** 8.2')



> Entering new AgentExecutor chain...
I need to calculate 3.14 raised to the power of 8.2 to get the answer.
Action: Python_REPL
Action Input: print(3.14 ** 8.2)
Observation: 11880.182554575464

Thought:I now know the final answer
Final Answer: 11880.182554575464

> Finished chain.


{'input': 'What is the asnwer to 3.14 ** 8.2', 'output': '11880.182554575464'}

In [19]:
response = agent.invoke('What is the asnwer to 3.14 ** 8.2')



> Entering new AgentExecutor chain...
I need to calculate 3.14 raised to the power of 8.2 to get the answer.
Action: Python_REPL
Action Input: print(3.14 ** 8.2)
Observation: 11880.182554575464

Thought:I now know the final answer
Final Answer: 11880.182554575464

> Finished chain.


In [20]:
print(response['input'])

What is the asnwer to 3.14 ** 8.2


In [21]:
print(response['output'])

11880.182554575464


### Create ReAct Agent

In [22]:
pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_openai import ChatOpenAI

In [24]:
llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)

In [25]:
template = '''
Answer the following question as best you can.
Questions: {question}
'''

In [26]:
prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')

### Python REPL Tool

In [27]:
python_repl = PythonAstREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func= python_repl.run,
    description='Use when you need to run Python to answer a question. You Should input Python code.'
)

### Wikipedia Tool

In [28]:
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func= wikipedia.run,
    description='Use for when you need to look up a topic, country, or person on Wikipedia.'
)

### DuckDuckGo Search

In [29]:
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func= search.run,
    description='Use for when you need to perform an internet search to find information that another tool can not provide.'
)

In [30]:
tools = [python_repl_tool, wikipedia_tool,duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools = tools,
    verbose = True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [31]:
question = 'What are the next 10 numbers in the Fibonacci sequence starting from 5'
output = agent_executor.invoke({'input': prompt_template.format(question=question)})



> Entering new AgentExecutor chain...
To find the next 10 numbers in the Fibonacci sequence starting from 5, I need to know the number that comes before 5 in the sequence. The Fibonacci sequence starts with 0, 1, and each subsequent number is the sum of the previous two. I can then calculate the next 10 numbers.

Action: Python REPL
Action Input:
```python
# Starting from the beginning of the Fibonacci sequence to find the number before 5
a, b = 0, 1
while b < 5:
    a, b = b, a + b
print(a, b)  # This should print the two numbers before calculating the next 10 numbers
```3 5
The number before 5 in the Fibonacci sequence is 3. Now, I can calculate the next 10 numbers in the sequence starting from 5.

Action: Python REPL
Action Input:
```python
# Starting from 3 and 5 to calculate the next 10 numbers in the Fibonacci sequence
a, b = 3, 5
sequence = [b]
for _ in range(10):
    a, b = b, a + b
    sequence.append(b)
print(sequence)
```[5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]
I now

In [32]:
question = 'What new scientific discoveries have been announced this week?'
output = agent_executor.invoke({'input': prompt_template.format(question=question)})



> Entering new AgentExecutor chain...
To find the most recent scientific discoveries, a search on the internet would provide the latest news and updates.

Action: DuckDuckGo Search
Action Input: new scientific discoveries this weekworld view | 21 Feb 2024 News Super-speedy sequencing puts genomic diagnosis in the fast lane Streamlined workflows for DNA and RNA sequencing are helping clinicians to deliver prompt, targeted... Science News | The latest news from all areas of science Astronomy JWST spies hints of a neutron star left behind by supernova 1987A Signs of highly ionized atoms in dusty clouds at SN... Mind How newly discovered brain cells have made us rethink the human mind. Strange new types of cells keep coming to light in the human brain. By the latest count, there are more than 3300, and we ... Feb. 20, 2024 — A few years ago, astronomers uncovered one of the Milky Way's greatest secrets: an enormous, wave-shaped chain of gaseous clouds in our sun's backyard, giving birth 

In [33]:
question = 'Tell me about Thomas Edison early life.'
output = agent_executor.invoke({'input': prompt_template.format(question=question)})



> Entering new AgentExecutor chain...
To find detailed information about Thomas Edison's early life, I will use Wikipedia as it usually provides comprehensive biographical details.
Action: Wikipedia
Action Input: Thomas EdisonPage: Thomas Edison
Summary: Thomas Alva Edison (February 11, 1847 – October 18, 1931) was an American inventor and businessman. He developed many devices in fields such as electric power generation, mass communication, sound recording, and motion pictures. These inventions, which include the phonograph, the motion picture camera, and early versions of the electric light bulb, have had a widespread impact on the modern industrialized world. He was one of the first inventors to apply the principles of organized science and teamwork to the process of invention, working with many researchers and employees. He established the first industrial research laboratory.Edison was raised in the American Midwest. Early in his career he worked as a telegraph operator, which i